In [172]:
import pandas as pd

In [173]:
# read datast(s)
with open('../data/Coaches.csv', 'rb') as f:
    coaches = pd.read_csv(f).fillna(0)

with open('../data/stadiums-geocoded.csv', 'rb') as f:
    stadium = pd.read_csv(f).fillna(0)

with open('../data/graduation-rates.csv', 'rb') as f:
    grad_rates = pd.read_csv(f).fillna(0)

with open('../data/2017-season.csv', 'rb') as f:
    season_2017 = pd.read_csv(f).fillna(0)

In [174]:
# define monetary columns
monetary_columns = [
    'SchoolPay',
    'TotalPay',
    'Bonus',
    'BonusPaid',
    'AssistantPay',
    'Buyout'
]

# convert lowercase
coaches = coaches.apply(lambda x: x.astype(str).str.lower())
stadium = stadium.apply(lambda x: x.astype(str).str.lower())
grad_rates = grad_rates.apply(lambda x: x.astype(str).str.lower())
season_2017 = season_2017.apply(lambda x: x.astype(str).str.lower())

# remove non-numeric to empty space
coaches[monetary_columns] = coaches[monetary_columns].replace('[\$,--]', '', regex=True)

# coerce monetary to numeric
coaches[monetary_columns] = coaches[monetary_columns].apply(pd.to_numeric, errors='coerce', axis=1).fillna(0)

# column names to lowercase
coaches.columns = coaches.columns.str.lower()
stadium.columns = stadium.columns.str.lower()
grad_rates.columns = grad_rates.columns.str.lower()
season_2017.columns = season_2017.columns.str.lower()

In [175]:
# convert column name: allows dataframe merge
stadium = stadium.rename(columns={'team': 'school'})

# stadium only columns
stadium = stadium[['stadium', 'school', 'capacity', 'built', 'expanded']]

# replace acronym with school name
stadium['school'].replace(['ucf'], 'central florida')
stadium['school'].replace(['usf'], 'south florida')
stadium['school'].replace(['utsa'], 'texas-san antonio')
stadium['school'].replace(['byu'], 'brigham young')
stadium['school'].replace(['utep'], 'texas-el paso')
stadium['school'].replace(['ucf'], 'central florida')
stadium['school'].replace(['tcu'], 'texas christian')
stadium['school'].replace(['unlv'], 'nevada-las vegas')
stadium['school'].replace(['smu'], 'southern methodist')
stadium['school'].replace(['niu'], 'northern illinois')
stadium['school'].replace(['miami (oh)'], 'miami (ohio)')
stadium['school'].replace(['fiu'], 'florida international')
stadium['school'].replace(['umass'], 'massachusetts')
stadium['school'].replace(['yale bulldogs'], 'connecticut')

# merge coaches + stadium
merged_df = pd.merge(coaches, stadium, on='school', how='inner')

# merge graduation rates
merged_df = pd.merge(merged_df, grad_rates, on='school', how='inner')

# merge 2017 season
merged_df = pd.merge(merged_df, season_2017, on='school', how='inner')

ValueError: can not merge DataFrame with instance of type <class 'pandas.core.indexes.base.Index'>

Index(['school', 'win', 'loss', 'pct'], dtype='object')
